In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from datasets import load_dataset
from tqdm.notebook import tqdm
from safetensors.torch import safe_open
from torch.utils.data import DataLoader
import os
from safetensors.torch import save_file
from torch.nn.utils.rnn import pad_sequence
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [2]:
def load_list_of_logits_safetensor(file_path):
    with safe_open(file_path, framework="pt") as f:
        logits_list = []
        for key in f.keys():
            logits_list.append(f.get_tensor(key))
    
    return logits_list

In [3]:
logits = load_list_of_logits_safetensor("llama-3.1-405b-gsm8k-reverse-KL-prompt-tensors.safetensors")

In [17]:
logits2 = load_list_of_logits_safetensor("llama-3.1-405b-gsm8k-base-tensors.safetensors")
logits2 = logits2[:2638]
print(len(logits2))

2638


In [8]:
def compute_next_token_confidence(logits_list, sample_step=1):
    confidences = []
    # Generate indices to process based on sample_step
    indices_to_process = range(0, len(logits_list), sample_step)
    # Initialize tqdm progress bar
    for idx in tqdm(indices_to_process, desc='Processing logits'):
        logits = logits_list[idx]
        # Move logits to GPU
        logits = logits.to('cuda')
        # Apply softmax to get probabilities
        probs = F.softmax(logits, dim=-1)
        # Get the maximum probability at each position (confidence)
        max_probs, _ = torch.max(probs, dim=-1)
        # Move back to CPU and convert to numpy if needed
        max_probs = max_probs.cpu().numpy()
        confidences.append(max_probs)
        # Clean up to free GPU memory
        del logits, probs, max_probs
        torch.cuda.empty_cache()
    return confidences

In [12]:
def compute_average_confidence(confidences):
    total_confidence = 0.0
    total_count = 0
    for max_probs in confidences:
        total_confidence += np.sum(max_probs)
        total_count += max_probs.size  # Total number of elements
    average_confidence = total_confidence / total_count
    return average_confidence

In [15]:
confidences = compute_next_token_confidence(logits, 5)

Processing logits:   0%|          | 0/528 [00:00<?, ?it/s]

In [16]:
confidence = compute_average_confidence(confidences)
print(confidence)

0.5205


In [18]:
confidences2 = compute_next_token_confidence(logits2, 5)

Processing logits:   0%|          | 0/528 [00:00<?, ?it/s]

In [19]:
confidence2 = compute_average_confidence(confidences2)
print(confidence2)

0.573
